In [1]:
import numpy as np
import torch
from torch import nn
torch.manual_seed(42)
from utils import train
from torch.utils.data import Dataset,DataLoader
import wandb

In [2]:
train_set=np.load("dataset/train.npz")
val_set=np.load("dataset/val.npz")

In [3]:
class Mydataset(Dataset):
    def __init__(self,dataset):
        self.X=torch.tensor(dataset['x'],dtype = torch.float32)
        self.y=torch.tensor(dataset['y'],dtype = torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,index):
        return self.X[index],self.y[index]

def generate_dataloaders(train_data,val_data,batch_size):
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    X, y=next(iter(train_loader))
    print(f'TRAIN: the shape of X: {X.shape}; the shape of y: {y.shape}')

    val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)
    X, y=next(iter(val_loader))
    print(f'VAL: the shape of X: {X.shape}; the shape of y: {y.shape}')
    return train_loader, val_loader

In [4]:
train_data = Mydataset(train_set)
val_data = Mydataset(val_set)
train_loader, val_loader=generate_dataloaders(train_data, val_data,batch_size=32)

TRAIN: the shape of X: torch.Size([32, 8, 49]); the shape of y: torch.Size([32, 1])
VAL: the shape of X: torch.Size([32, 8, 49]); the shape of y: torch.Size([32, 1])


In [5]:
class RNNModel(nn.Module):
    """The RNN model."""
    def __init__(self, input_dimension,output_dimension,num_layers,num_hiddens):
        super().__init__()
        self.input_dimension = input_dimension
        self.output_dimension=output_dimension
        self.num_hiddens=num_hiddens
        self.rnn = nn.LSTM(self.input_dimension, self.num_hiddens,num_layers)
        self.linear = nn.Linear(self.num_hiddens, self.output_dimension)

    def forward(self, inputs,state=None):
        X = inputs.permute(1,0,2)
        X = X.to(torch.float32)
        _, state = self.rnn(X)
        # if state is not None:
        #     _,state = self.rnn(X, state)
        # else:
        #     _, state = self.rnn(X)
        output = self.linear(state[-1][-1]) # take the hidden state 2, on the layer 2
        return output
    
    # def begin_state(self, device, batch_size=1):
    #     if not isinstance(self.rnn, nn.LSTM):
    #         # `nn.GRU` takes a tensor as hidden state
    #         return torch.zeros((self.rnn.num_layers,
    #                             batch_size, self.num_hiddens), device=device)
    #     else:
    #         # `nn.LSTM` takes a tuple of hidden states
    #         return (torch.zeros((self.rnn.num_layers,
    #                              batch_size, self.num_hiddens),
    #                             device=device),
    #                 torch.zeros((self.rnn.num_layers,
    #                              batch_size, self.num_hiddens),
    #                             device=device))

In [6]:
use_wandb=False
net = RNNModel(input_dimension=49,output_dimension=1,num_layers=2,num_hiddens=8)
num_epochs=5
load_from_path=None
save_to_path="checkpoints/v0/"
if use_wandb:
    wandb.login()
    wandb.init(project="trial", config={'lr':0.001,'num_layers':2,'num_hiddens':8})

In [7]:
# Trail
X_sample, _ =next(iter(train_loader))
y_trail=net(X_sample)

In [9]:
y_trail.shape

torch.Size([32, 1])